### Модуль предобработки данных. 

In [1]:
#pip install scikit-learn==1.1
#%pip install --upgrade scikit-learn==1.4.0

In [39]:
import pandas as pd
import os
import pickle

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

##### Имитируем подачу данных с датчиков

In [40]:
# train_FD001 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/train_FD001.csv")
# train_FD002 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/train_FD002.csv")
# train_FD003 = pd.read_csv("C:/Users\Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/train_FD003.csv")
# train_FD004 = pd.read_csv("C:/Users\Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/train_FD004.csv")

# test_FD001 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/test_FD001.csv")
# test_FD002 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/test_FD002.csv")
# test_FD003 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/test_FD003.csv")
# test_FD004 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/test_FD004.csv")

#This dataset using for show preprocess handling incoming data
#BAD_test_FD001 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/BAD_test_FD001.csv")

Small_BAD_test_FD001 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/Small_BAD_test_FD001.csv")
Small_GOOD_test_FD001 = pd.read_csv("C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/Small_GOOD_test_FD001.csv")

##### Проверим "плохой" датасет

In [35]:
Small_GOOD_test_FD001

,unit number,time in cycles,operational setting 1,operational setting 2,Unnamed: 4
0,1,1,0.0023,0.0003,100.0
1,1,2,-0.0027,-0.0003,100.0
2,1,3,0.0003,0.0001,100.0


In [36]:
Small_BAD_test_FD001

,unit number,time in cycles,operational setting 1,operational setting 2,Unnamed: 4
0,1,1.0,0.0023,NaN,NaN
1,1,NaN,-0.0027,-0.0003,100.0
2,1,3.0,NaN,0.0001,100.0


In [ ]:
null_set = pd.Dadataframe(Small_test_FD001.isnull())
null_set

Поиск пропусков

In [ ]:
def is_null(dataframe: pd.DataFrame):
    count_nulls = 0
    null_set = pd.Dadataframe(dataframe.isnull())

    for col in dataframe.columns:
        for val in dataframe[col].is_null():
            if (val == True):
                count_nulls +=1
            #print(val)
        print("in ", col, "nulls: ", count_nulls)
        count_nulls = 0

is_null(Small_test_FD001)

##### Создаем SimpleInputer - простой заполнитель пропущенных значений в датасете

Так как предполагается, что данные упорядоченны по циклам работы двигателя, не целесообразно использовать стандартный заполнитель SimpleImputer. разумнее будет вычислить среднее значение лишь до и после пропущенного значения в каждом столбце. Можно использовать KNNImputer.

In [65]:
simple_inputer = KNNImputer(n_neighbors = 2)
std_scaler = StandardScaler()

pipe_num = Pipeline([('imputer', simple_inputer),('scaler', std_scaler)])

Обучаем Pipeline

In [70]:
res_num = pipe_num.fit_transform(Small_GOOD_test_FD001)

Теперь можем применить полученный pipeline
Видим, что пропусков нет

In [67]:
res_num = pd.DataFrame(res_num, columns=pipe_num['scaler'].get_feature_names_out(Small_GOOD_test_FD001.columns))
res_num

,unit number,time in cycles,operational setting 1,operational setting 2,Unnamed: 4
0,0.0,-1.224745,1.135550,1.069045,0.0
1,0.0,0.000000,-1.297771,-1.336306,0.0
2,0.0,1.224745,0.162221,0.267261,0.0


In [68]:
Small_GOOD_test_FD001

,unit number,time in cycles,operational setting 1,operational setting 2,Unnamed: 4
0,1,1,0.0023,0.0003,100.0
1,1,2,-0.0027,-0.0003,100.0
2,1,3,0.0003,0.0001,100.0


Сохраняем Pipline

In [18]:
savePath = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/final/Pipelinr.pickle'

with open(savePath, 'wb') as handle:
    save_pik_pipeline = pickle.dumps(pipe_num)

In [18]:
inp_dir = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/processed'
save_dir = f"{inp_dir}/"
files = os.listdir(inp_dir)

files

['new_1.txt']

In [4]:
for file in files:
    new_name = f'{save_dir}new_{file}'
    os.rename(save_dir+file, new_name)

In [6]:
if save_dir[-1] == '/':
    print("eeee")

eeee


Сохраняем pickle

In [85]:
def employ_Pipline(
                        inp_dir: str,
                        out_dir: str,
                        pipline: Pipeline):
        
        status_log = ["Preprocess data finished successfull", "Preprocess data finished error"]

        # Получаем все документы в папке
        inpFilesList = os.listdir(inp_dir)
        outFilesList = inpFilesList
        
        for fl in inpFilesList:
            print(fl)

        # На конце выходной строки дирректории должна стоять "/"
        if inp_dir[-1] != '/':
            inp_dir = f"{inp_dir}/"
        if out_dir[-1] != '/':
            out_dir = f"{out_dir}/"

        # Применение
        for file in inpFilesList:
            if file == ".gitkeep":
                 continue
                 
            print("    Processed -> ", file)
            dataFrame = pd.read_csv(inp_dir + file)
            #pipline.fit_transform(file)
            newDataFrame = pd.DataFrame(dataFrame, columns=pipline['scaler'].get_feature_names_out(dataFrame.columns))

            filename, extension = os.path.splitext(file)
            newDataFrame.to_pickle(f"{out_dir}new_{filename}.pickle")
        
        print(status_log[0])
        return 0

In [87]:
inp_dir = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/tests'
out_dir = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/processed'

employ_Pipline(inp_dir,
               out_dir,
               pipe_num)

Small_BAD_test_FD001.csv
Small_GOOD_test_FD001.csv
    Processed ->  Small_BAD_test_FD001.csv
    Processed ->  Small_GOOD_test_FD001.csv
Preprocess data finished successfull


0

### Источники
1 - https://habr.com/ru/companies/yandex_praktikum/articles/756474/